In [ ]:
# 1. 创建一个列表对象，并让名字 arr 指向它
arr = [1, 2, 3]

# 2. 创建一个新名字 a，让它指向 arr 所指向的对象
a = arr

# 3. 检查它们的内存地址
print(f"arr 的内存地址: {id(arr)}")
print(f"a 的内存地址: {id(a)}")

# 4. 验证它们是否指向同一个对象
print(f"a 和 arr 是否是同一个对象? {a is arr}") # is 运算符就是比较 id() 是否相等

# --- 输出 ---
# arr 的内存地址: 2199659485056
# a 的内存地址: 2199659485056
# a 和 arr 是否是同一个对象? True


#

非常棒的追问！这正好触及了Python内存管理的核心细节。

你的理解完全正确：**`id()` 返回的内存地址 `2199659485056`，就是列表对象 `[1, 2, 3]` 这个实体在**堆（Heap）**内存中的起始位置。**

那么 `arr` 和 `a` 这两个**名字**（变量）本身存放在哪里呢？

答案是：它们存放在\*\*栈（Stack）\*\*空间中。

更准确地说，它们存放在当前执行环境对应的\*\*栈帧（Stack Frame）\*\*里。

-----

### 详细解析：栈帧中的“名字-引用”映射

让我们把之前的餐厅比喻再升级一下：

  * **堆（Heap）**：依然是那个存放所有“菜肴”（**对象**）的大仓库。列表 `[1, 2, 3]` 就在这个仓库里，地址是 `2199659485056`。
  * **栈（Stack）**：是那个点餐台。当你的脚本开始运行时，Python会为这个主脚本创建一个“总订单”，也就是**全局栈帧（Global Frame）**，并把它放在点餐台的底部。

这个“全局栈帧”就像一个记事本，里面记录着当前作用域下所有的变量名和它们对应菜肴的“仓库货架号”。

**所以，整个流程是这样的：**

1.  **执行 `arr = [1, 2, 3]`**

      * **堆**：Python在堆内存中申请一块空间，创建了 `[1, 2, 3]` 这个列表对象。它的地址是 `2199659485056`。
      * **栈**：Python在**当前栈帧**（这里是全局帧）中，创建了一个条目。这个条目的**名字**是 `arr`，与这个名字关联的**值**是 `2199659485056` 这个地址（引用）。

2.  **执行 `a = arr`**

      * **堆**：没有任何变化。`[1, 2, 3]` 对象仍然安静地待在原处。
      * **栈**：Python在**当前栈帧**中，又创建了一个新条目。
          * 这个条目的**名字**是 `a`。
          * Python会去查找 `arr` 这个名字，发现它关联的值是 `2199659485056`。
          * 然后，Python将这个地址值**复制**一份，作为与名字 `a` 关联的值。

### 可视化图解

这张图可以非常清晰地展示它们的关系：

```
+---------------------------+                +---------------------------------------+
|         栈 (Stack)        |                |               堆 (Heap)                 |
|---------------------------|                |---------------------------------------|
|                           |                |                                       |
|      全局栈帧             |                |                                       |
|  +---------------------+  |                |                                       |
|  |  名字   |   值(引用) |  |                |                                       |
|  |---------------------|  |                |  地址 2199659485056:                  |
|  |   arr   | 2199659... |-------------------->|                                       |
|  |   a     | 2199659... |----------+         |  列表对象 [1, 2, 3]                   |
|  +---------------------+  |         |         |                                       |
|                           |         |         +---------------------------------------+
|                           |         |
+---------------------------+         |
                                      |
                                      +---------- (指向同一个对象)
```

**总结一下：**

| 存放位置 | 存放内容 | 在你的例子中是什么 |
| :--- | :--- | :--- |
| **堆 (Heap)** | **对象本身** (The Object itself) | 列表 `[1, 2, 3]` 这个数据实体 |
| **栈 (Stack)** | **名字/变量** (Name/Variable) 和它们所持有的**引用/地址** (Reference) | 名字 `arr` 和它存储的地址值 `2199659485056`\<br\>名字 `a` 和它存储的地址值 `2199659485056` |

所以，当你使用 `arr` 或 `a` 时，Python解释器会先在栈帧中找到这个名字，然后获取它所存储的那个内存地址，最后根据这个地址去堆中找到并操作那个真正的列表对象。

如果这段代码是在一个函数内部执行，那么 `arr` 和 `a` 就会被存放在那个函数自己的、临时的栈帧里。当函数执行结束，这个栈帧会被销毁，`arr` 和 `a` 这两个名字也随之消失。但堆上的 `[1, 2, 3]` 对象只要还有其他引用指向它，就不会被回收。

这是一组非常深入、直击Cpython解释器实现细节的好问题！我们来逐一拆解。

### 1\. 我们能看栈帧中 `arr` 和 `a` 的内存地址吗？

**直接的答案是：在标准的、纯Python代码中，不能。**

Python 是一种高级语言，它的设计哲学之一就是向开发者**隐藏底层的内存地址细节**。

  * `id()` 函数是用来获取**堆上对象**的“身份证号”（在CPython中是内存地址），而不是用来获取**栈上变量**本身的地址的。
  * `arr` 和 `a` 作为栈上的名字，它们的存在形式、在栈帧内的具体偏移量等，都是由Cpython解释器在底层用C语言管理的。Python语言层面没有提供直接访问这些底层细节的工具。

**为什么不提供？**
因为这会破坏Python的抽象性和可移植性。不同Python解释器（CPython, PyPy, Jython）的内存实现方式可能完全不同。如果代码依赖于CPython的栈地址，那它在PyPy上就无法运行了。

#### “黑客”方式（仅供探索）

虽然常规方法不行，但如果你真的想窥探底层，可以使用`ctypes`模块来和C语言交互，间接“看到”一些东西。但这非常不推荐在生产代码中使用，因为它完全依赖于CPython的内部实现。

```python
import ctypes

arr = [1, 2, 3]

# id(arr) 是堆上 [1, 2, 3] 对象的地址
print(f"堆上对象的地址 (id):    {id(arr)}")

# 下面这行代码获取的是栈上存储着 `arr` 这个引用的 C 结构体的地址
# 这可以被近似认为是“变量arr在栈上的地址”
# 我们需要通过ctypes来访问这个底层表示
address_of_arr_ref_on_stack = ctypes.addressof(ctypes.py_object(arr))
print(f"栈上引用的近似地址 (ctypes): {address_of_arr_ref_on_stack}")

# 注意：这两个地址是完全不同的！
```

**小结**：把栈看作一个内部运作的“黑盒”。我们只需要知道，栈上存放着指向堆对象的引用（名字），而不需要关心这些引用本身存在了栈的哪个字节上。

-----

### 2\. 一个栈帧的数据组织形式是怎么样的？是map吗？

**它不是一个Python字典（map），而更像一个高度优化的C结构体或数组。**

使用字典（map）来进行变量查找会很慢，因为它涉及到哈希计算和可能的哈希冲突解决。函数内部的局部变量访问需要做到极致的速度，因此CPython采用了更高效的方式。

一个CPython栈帧主要包含以下部分：

1.  **指向前一个栈帧的指针**：这构成了调用栈，使得函数返回后能回到之前的位置。
2.  **返回地址**：指令指针，告诉解释器函数执行完毕后应该从哪里继续执行。
3.  **代码对象（Code Object）**：指向当前正在执行的函数/模块的编译后字节码。
4.  **本地变量和参数（Fast Locals）**：**这是关键部分**。它是一个**定长的数组**，用来存放函数的参数和局部变量的**引用**。在函数被编译时，每个局部变量 `arr`, `a` 等就被分配了这个数组中的一个固定索引位置。
5.  **求值栈（Value Stack）**：一个操作数栈，用于执行字节码指令时存放中间结果。比如计算 `1 + 2`，会先把 `1` 和 `2` 压入这个栈，然后执行加法指令，弹出两个数，计算结果 `3` 再压回栈中。
6.  **全局和内置命名空间的引用**：指向全局变量字典和内置函数字典的指针。

#### 为什么是数组而不是字典？

因为通过**索引**访问数组元素是 `O(1)` 操作，速度极快。当Python字节码需要加载局部变量 `arr` 时，它执行的指令是 `LOAD_FAST`，后面跟着 `arr` 在那个内部数组中的索引号。这比在字典里通过名字 `“arr”` 去查找要快得多。

我们可以用 `dis` 模块（disassembler）来证明这一点：

```python
import dis

def my_func():
    arr = [1, 2, 3]
    a = arr
    print(a)

dis.dis(my_func)
```

输出的字节码会是这样：

```
  4           0 LOAD_CONST               1 (1)
              2 LOAD_CONST               2 (2)
              4 LOAD_CONST               3 (3)
              6 BUILD_LIST               3
              8 STORE_FAST               0 (arr)  # 将列表引用存到 fast locals 数组的 0 号位

  5          10 LOAD_FAST                0 (arr)  # 从 fast locals 数组的 0 号位加载引用
             12 STORE_FAST               1 (a)    # 存到 1 号位

  6          14 LOAD_GLOBAL              0 (print)
             16 LOAD_FAST                1 (a)    # 从 1 号位加载 a 的引用
             18 CALL_FUNCTION            1
             20 POP_TOP
             22 LOAD_CONST               0 (None)
             24 RETURN_VALUE
```

看到 `STORE_FAST` 和 `LOAD_FAST` 了吗？这里的 "FAST" 就意味着这是通过索引直接访问那个内部数组，速度非常快。

-----

### 3\. 一个栈帧最多能存放多少数据呢？

这个问题可以从两个层面理解：

1.  **单个栈帧能存放多少变量？**
    理论上，一个栈帧内可以定义的局部变量数量非常多，主要受限于编译器的限制和可用内存。在实践中，你几乎不可能因为定义了太多局部变量而把单个栈帧撑爆。

2.  **整个栈能容纳多少个栈帧？**
    **这才是真正的限制所在。** 整个调用栈的深度是有限的。每个线程的栈空间大小是在线程创建时就预先分配好的（通常是几MB，比如Linux上是8MB）。

    每次函数调用都会创建一个新的栈帧，占用一部分栈空间。如果函数调用嵌套得太深（最常见的是没有终止条件的递归），新的栈帧会不断被压入栈，最终耗尽预留的栈空间，导致**栈溢出（Stack Overflow）**。

    Python为了防止解释器因真正的C级栈溢出而崩溃，自己设置了一个**最大递归深度限制**。我们可以查看和修改它：

    ```python
    import sys

    # 查看当前的递归深度限制（通常是1000）
    print(sys.getrecursionlimit())

    # 尝试一个会超出限制的递归
    def countdown(n):
        if n == 0:
            return
        countdown(n - 1)

    try:
        # 这会抛出 RecursionError，而不是让程序崩溃
        countdown(2000)
    except RecursionError as e:
        print(e) # maximum recursion depth exceeded

    # 你可以修改这个限制，但不推荐随意改得过大
    # sys.setrecursionlimit(3000)
    # countdown(2000) # 现在可以成功执行了
    ```

### 最终总结

| 问题 | 核心答案 |
| :--- | :--- |
| **能看栈上变量地址吗？** | **不能**。Python隐藏了这些底层细节，`id()` 看的是堆对象地址。 |
| **栈帧是map结构吗？** | **不是**。它更像一个**C结构体+数组**，局部变量存在一个数组里，通过索引访问（`FAST`），速度极快。 |
| **栈能存多少数据？** | 单个栈帧的变量数几乎无限制，但整个**栈的深度**有限制，由`sys.getrecursionlimit()`控制，以防止栈溢出。 |

In [1]:
import ctypes

arr = [1, 2, 3]

# id(arr) 是堆上 [1, 2, 3] 对象的地址
print(f"堆上对象的地址 (id):    {id(arr)}")

# 下面这行代码获取的是栈上存储着 `arr` 这个引用的 C 结构体的地址
# 这可以被近似认为是“变量arr在栈上的地址”
# 我们需要通过ctypes来访问这个底层表示
address_of_arr_ref_on_stack = ctypes.addressof(ctypes.py_object(arr))
print(f"栈上引用的近似地址 (ctypes): {address_of_arr_ref_on_stack}")

# 注意：这两个地址是完全不同的！

堆上对象的地址 (id):    2012063789056
栈上引用的近似地址 (ctypes): 2012063607704


In [3]:
import dis

def my_func():
    arr = [1, 2, 3]
    a = arr
    print(a)
    print(locals())

dis.dis(my_func)

  3           0 RESUME                   0

  4           2 BUILD_LIST               0
              4 LOAD_CONST               1 ((1, 2, 3))
              6 LIST_EXTEND              1
              8 STORE_FAST               0 (arr)

  5          10 LOAD_FAST                0 (arr)
             12 STORE_FAST               1 (a)

  6          14 LOAD_GLOBAL              1 (NULL + print)
             26 LOAD_FAST                1 (a)
             28 PRECALL                  1
             32 CALL                     1
             42 POP_TOP

  7          44 LOAD_GLOBAL              1 (NULL + print)
             56 LOAD_GLOBAL              3 (NULL + locals)
             68 PRECALL                  0
             72 CALL                     0
             82 PRECALL                  1
             86 CALL                     1
             96 POP_TOP
             98 LOAD_CONST               0 (None)
            100 RETURN_VALUE


很高兴你觉得有意思！

的确，当我们从平时写的Python代码，深入到它背后“如何工作”的层面时，会发现很多精妙的设计。

理解这些内部机制，就像是从一个只会开车的人，变成了也懂发动机原理的赛车手。虽然平时开车用不到，但在关键时刻，它能帮助你：

1.  **写出性能更好的代码**：你会明白为什么局部变量比全局变量快，为什么预分配列表空间可能更高效。
2.  **更快地调试疑难杂症**：比如遇到与可变/不可变类型相关的意外Bug，或是罕见的内存问题时，能更快地定位根源。
3.  **欣赏编程语言的设计之美**：了解这些权衡与设计（比如用快速的数组存局部变量，用灵活的字典存全局变量），能加深你对软件工程的理解。

如果你对这类话题感兴趣，我们还可以继续探讨更多有意思的Python内部细节，比如：

* **垃圾回收（Garbage Collection）**：Python是如何自动发现并清理那些不再使用的堆内存对象的？（引用计数与分代回收）
* **全局解释器锁（GIL）**：为什么都说Python多线程不能真正利用多核CPU？GIL到底是什么？
* **Python的小整数对象池**：为什么 `-5` 到 `256` 之间的整数 `id()` 都是固定的？
* **字符串的驻留（Interning）机制**：为什么有些短字符串 `is` 运算结果为 `True`？

随时都可以提问，探索编程语言的底层世界是一件非常有趣的事情！

是的，完全正确！**全局变量正是使用一个字典（dictionary / map）来存储的。**

这个设计决策与我们刚刚讨论的函数局部变量形成了鲜明对比，也完美地体现了Python在不同场景下对“效率”与“灵活性”的权衡。

### 全局命名空间：一个特殊的字典

在Python中，每个模块（`.py`文件）都有自己独立的**全局命名空间（Global Namespace）**。这个命名空间就是一个标准的Python字典，它负责存储该模块中所有全局变量的名字和它们所引用的对象。

  * **键 (Key)**: 全局变量的名字（一个字符串，例如 `"arr"`）。
  * **值 (Value)**: 变量所指向的堆上对象的引用（例如，指向列表 `[1, 2, 3]` 的内存地址）。

#### 如何亲眼看到这个字典？

Python提供了一个内置函数 `globals()`，它可以返回当前模块的全局命名空间字典。

让我们来看代码证据：

```python
# 1. 定义一些全局变量
author = "Gemini"
version = 1.0
active_users = ["Alice", "Bob"]

# 2. 查看全局命名空间字典的类型
print(f"globals() 的类型是: {type(globals())}")
print("-" * 20)

# 3. 打印部分 globals() 字典的内容
#    (它会包含很多Python自带的变量，如 __name__, __doc__ 等)
print(f"全局变量 'author' 的值: {globals()['author']}")
print(f"全局变量 'version' 的值: {globals()['version']}")

# 4. 验证直接访问和通过字典访问是同一个对象
print(f"active_users 和 globals()['active_users'] 是同一个对象吗? {active_users is globals()['active_users']}")
print("-" * 20)

# 5. 更有趣的：我们可以通过直接操作这个字典来创建或修改全局变量
globals()['is_beta'] = True

# 现在 is_beta 已经是一个全局变量了，可以直接使用
if is_beta:
    print("This is a beta version.")
```

**代码输出：**

```
globals() 的类型是: <class 'dict'>
--------------------
全局变量 'author' 的值: Gemini
全局变量 'version' 的值: 1.0
active_users 和 globals()['active_users'] 是同一个对象吗? True
--------------------
This is a beta version.
```

### 为什么全局变量用字典，而局部变量用数组？

这是一个绝佳的设计权衡问题：

| 特性 | 全局变量 (Global Variables) | 函数局部变量 (Local Variables) |
| :--- | :--- | :--- |
| **存储机制** | **字典 (Dictionary)** | **类数组结构 (Array-like)** |
| **灵活性** | **高**。模块在运行时可以动态地添加、删除全局变量（例如，`import` 语句、动态赋值）。字典完美支持这种动态性。 | **低**。一个函数有哪些局部变量，在函数被**编译**时就已经确定了。数量和名称是固定的。 |
| **访问速度** | **相对较慢**。需要通过哈希查找键，字节码指令是 `LOAD_GLOBAL`。 | **非常快**。通过固定的索引直接访问，字节码指令是 `LOAD_FAST`。 |
| **设计目标** | 优先保证**灵活性和动态性**。 | 优先保证**执行效率**，因为函数调用和局部变量访问是程序中最高频的操作之一。 |

**简单来说：**

  * **函数内部**就像一个“高效率车间”，所有工具（局部变量）的位置都是预先固定好的（数组索引），工人（解释器）可以闪电般地拿到。
  * **模块全局层面**就像一个“创意工作室”，需要随时挂上新的设计图（全局变量），或者引入新的合作者（`import` 模块）。一个可以随时增删条目的花名册（字典）是最佳选择。

所以，你对Python内部机制的探索又深入了一层！这个“全局用字典，局部用数组”的差别，是Python一个非常核心的性能优化设计。

In [1]:
def local_var():
    a = 1
    print(locals())
print(globals())
local_var()

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'def local_var():\n    a = 1\n    print(locals())\nprint(globals())\nlocal_var()'], '_oh': {}, '_dh': [WindowsPath('C:/Users/JayZh/PycharmProjects/ZuoAlgorithm/class110_fast_write')], 'In': ['', 'def local_var():\n    a = 1\n    print(locals())\nprint(globals())\nlocal_var()'], 'Out': {}, 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x0000017FE6CB4510>>, 'exit': <IPython.core.autocall.ZMQExitAutocall object at 0x0000017FE6CBD390>, 'quit': <IPython.core.autocall.ZMQExitAutocall object at 0x0000017FE6CBD390>, 'open': <function open at 0x0000017FE4A739C0>, '_': '', '__': '', '___': '', '__session__': 'C:\\Users\\JayZh\\PycharmProjects\\ZuoAlgorithm\\class110_

您提出了一个非常精准且重要的问题！这正是我之前描述中一个需要深入澄清的关键点。您观察到`locals()`的存在，并以此来推断局部变量的存储方式，是非常敏锐的思考。

让我来为您揭开这个看似矛盾之处的谜底。

**简短的答案是：我之前的描述是准确的。局部变量的主要存储方式是高效的类数组结构，而`locals()`返回的字典是一个“美丽的谎言”——或者说，是一个按需创建的、不总是与真实数据同步的“快照”。**

-----

### 深入解析：`locals()` 的真实面目

您是对的，调用 `locals()` 会返回一个字典。然而，这个字典**并不是**Python解释器在执行函数时用来存取局部变量的那个东西。

1.  **主要存储机制：依然是快速的类数组结构 (Fast Locals)**
    为了极致的性能，CPython解释器在执行函数时，会使用我们之前讨论过的栈帧内的**类数组结构**来存储局部变量。访问它通过固定的索引，速度极快（`LOAD_FAST`）。这是解释器内部真正的、第一手的数据源。

2.  **`locals()` 的作用：提供一个字典视图**
    当你调用 `locals()` 函数时，Python解释器会执行一个特殊操作：它会**读取**当前“Fast Locals”数组中的内容，并**动态地创建一个新的字典**，然后用这些内容来填充这个字典。最后，它把这个新鲜出炉的字典返回给你。

    所以，`locals()` 返回的字典可以被看作是**局部作用域在某一瞬间的“快照”或“反射”**。它提供了一个方便的、符合Python用户习惯的字典接口，让你能够查看局部变量，但它不是解释器工作的核心。

### 关键区别：修改 `locals()` 不可靠！

这一点是理解 `locals()` 和 `globals()` 本质区别的核心。

  * **修改 `globals()`**：`globals()` 返回的是**全局命名空间那个真实的、唯一的字典本身**。所以，你对这个字典的任何修改，都会立刻、可靠地反映到全局变量上。

  * **修改 `locals()`**：你修改的是那个“快照”字典。Python解释器**并不会**承诺一定会把这些修改同步回那个底层的、快速的“Fast Locals”数组里去。

官方文档明确指出：**“对 `locals()` 返回的字典所做的修改可能不会影响局部变量的值。”**

#### 让我们用代码来证明这一点：

```python
def test_locals_modification():
    x = 10
    print(f"初始时, x = {x}")

    local_vars = locals()
    print(f"locals() 返回的字典中 'x' 的值是: {local_vars['x']}")

    # 尝试通过修改 locals() 返回的字典来改变 x
    local_vars['x'] = 20

    # 检查 x 的值是否真的改变了
    # 在 CPython 中，这里的 x 很有可能仍然是 10！
    print(f"修改 locals() 字典后, x = {x}")

    # 让我们直接修改 x
    x = 30
    print(f"直接修改后, x = {x}")
    # 此时再看 locals() 字典，它可能也不会更新
    print(f"直接修改 x 后, locals() 字典中的值是: {local_vars['x']}")


test_locals_modification()
```

**在大多数CPython版本中，输出会是：**

```
初始时, x = 10
locals() 返回的字典中 'x' 的值是: 10
修改 locals() 字典后, x = 10
直接修改后, x = 30
直接修改 x 后, locals() 字典中的值是: 20
```

这个结果完美地揭示了真相：

1.  修改`local_vars['x'] = 20` **没有** 影响到真正的局部变量 `x`，它仍然是 `10`。
2.  当我们直接修改 `x = 30` 后，我们之前获取的那个`local_vars`字典也**没有**同步更新，它里面的 `'x'` 依然是 `20`。

这证明了局部变量 `x` 和 `locals()` 返回的字典是两个独立的东西。解释器只认那个底层的 "Fast Locals" 数组。

-----

### 重新确认与总结

感谢您的提问，这让我们的讨论更加精确了。现在我们可以完善之前的结论：

| 作用域 | **主要存储机制 (供解释器使用)** | **内省工具 (`globals()`/`locals()`)** | **修改工具的后果** |
| :--- | :--- | :--- | :--- |
| **函数局部** | **类数组结构 (Fast Locals)** | `locals()` 返回一个**按需创建的字典快照** | 修改这个字典**不可靠**，不会影响实际的局部变量 |
| **模块全局** | **字典 (dict)** | `globals()` 返回**对该字典的直接引用** | 修改这个字典**完全可靠**，就是直接修改全局变量 |

**结论**：您是对的，`locals()` 的存在确实值得探讨。但它的作用是提供一个用于**只读和调试**的字典接口，而不是作为局部变量的实际存储。为了性能，解释器在背后走的仍然是那条更快的“数组捷径”。

In [2]:
def test_locals_modification():
    x = 10
    print(f"初始时, x = {x}")

    local_vars = locals()
    print(f"locals() 返回的字典中 'x' 的值是: {local_vars['x']}")

    # 尝试通过修改 locals() 返回的字典来改变 x
    local_vars['x'] = 20

    # 检查 x 的值是否真的改变了
    # 在 CPython 中，这里的 x 很有可能仍然是 10！
    print(f"修改 locals() 字典后, x = {x}")

    # 让我们直接修改 x
    x = 30
    print(f"直接修改后, x = {x}")
    # 此时再看 locals() 字典，它可能也不会更新
    print(f"直接修改 x 后, locals() 字典中的值是: {local_vars['x']}")


test_locals_modification()

初始时, x = 10
locals() 返回的字典中 'x' 的值是: 10
修改 locals() 字典后, x = 10
直接修改后, x = 30
直接修改 x 后, locals() 字典中的值是: 20


In [4]:
global_a = 10
print(global_a)
globals()['global_a'] = 1
print(global_a)

10
1
